In [1]:
import os
os.environ['AWS_ACCESS_KEY_ID']='hive'
os.environ['AWS_SECRET_ACCESS_KEY']='#202203sdf'
os.environ['AWS_REGION']='us-east-1'
           
# spark-shell --packages org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:0.13.1,org.apache.iceberg:iceberg-hive-runtime:0.13.1,org.apache.hadoop:hadoop-aws:3.3.1,software.amazon.awssdk:bundle:2.15.40,software.amazon.awssdk:url-connection-client:2.15.40  --conf spark.sql.catalog.demo=org.apache.iceberg.spark.SparkCatalog    --conf spark.sql.catalog.demo.uri=thrift://localhost:9083  --conf spark.sql.catalog.demo.type=hive      --conf spark.sql.catalog.demo.io-impl=org.apache.iceberg.aws.s3.S3FileIO --conf  spark.sql.catalog.demo.warehouse=s3a://warehouse/   --conf spark.sql.catalog.demo.s3.endpoint=http://localhost:9000   --conf spark.sql.defaultCatalog=demo --conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem
# --conf spark.sql.catalog.spark_catalog.io-impl=org.apache.iceberg.aws.s3.S3FileIO
# --conf spark.sql.catalog.spark_catalog.s3.endpoint=http://minio:9000 
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:0.13.1,org.apache.iceberg:iceberg-hive-runtime:0.13.1,org.apache.hadoop:hadoop-aws:3.3.1,software.amazon.awssdk:bundle:2.15.40,software.amazon.awssdk:url-connection-client:2.15.40 --conf spark.sql.catalog.spark_catalog=org.apache.iceberg.spark.SparkSessionCatalog --conf spark.sql.catalog.spark_catalog.type=hive --conf spark.sql.catalog.spark_catalog.cache-enabled=false  --conf spark.sql.catalog.spark_catalog.warehouse=s3a://hive/ --conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem --conf spark.sql.catalog.spark_catalog.uri=thrift://hive-metastore:9083 --conf spark.sql.catalog.spark_catalog.hadoop.fs.s3a.endpoint=http://minio:9000 --conf spark.sql.catalog.spark_catalog.s3a.endpoint=http://minio:9000 --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions pyspark-shell'

from pyspark.sql import SparkSession

tableName = "tbltestpython"

# Enable SQL/DML commands and Metastore tables for the current spark session.
# We need to set the following configs

  
aws_key = 'hive'
aws_secret = '#202203sdf'
spark = SparkSession.builder.appName("quickstart_sql").master("local[*]") \
  .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
  .config("spark.hadoop.fs.s3a.path.style.access", True) \
  .config('spark.hadoop.fs.s3a.access.key', aws_key) \
  .config('spark.hadoop.fs.s3a.secret.key',  aws_secret) \
  .config('spark.hadoop.fs.s3a.endpoint', 'http://minio:9000') \
  .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
  .config("hive.metastore.warehouse.dir", "s3a://hive/") \
  .config("spark.sql.warehouse.dir", "s3a://hive/") \
  .config("spark.hadoop.hive.metastore.warehouse.dir", "s3a://hive/") \
  .enableHiveSupport().getOrCreate()



In [14]:
spark.sql("use spark_catalog.default")

DataFrame[]

In [2]:
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [5]:
spark.sql(f"drop table if exists {tableName}")

DataFrame[]

In [2]:
spark.sql(f"CREATE TABLE if not exists {tableName}(id LONG) USING iceberg location 's3a://hive/{tableName}'")

DataFrame[]

In [7]:
spark.sql(f"select * from {tableName}").show()

+---+
| id|
+---+
+---+



In [7]:
spark.sql("drop table if exists newData")

DataFrame[]

In [3]:
spark.sql("CREATE TABLE if not exists newData(id LONG) USING parquet")

AnalysisException: org.apache.hadoop.hive.ql.metadata.HiveException: MetaException(message:file:/user/hive/warehouse/newdata is not a directory or unable to create one)

In [4]:
spark.sql("show create table newData").show(10, False)

AnalysisException: Table or permanent view not found: newData; line 1 pos 18;
'ShowCreateTable false, [createtab_stmt#12]
+- 'UnresolvedTableOrView [newData], SHOW CREATE TABLE, false


In [4]:

try:
    # Create a table
    print("############# Creating a table ###############")
    spark.sql("drop table if exists %s" % tableName)
    spark.sql(f"CREATE TABLE if not exists {tableName}(id LONG) USING iceberg location 's3a://hive/{tableName}' ")
    spark.sql("truncate table %s" % tableName)
    spark.sql("INSERT INTO %s VALUES 0, 1, 2, 3, 4" % tableName)

    # Read the table
    print("############ Reading the table ###############")
    spark.sql("SELECT * FROM %s" % tableName).show()

    # Upsert (merge) new data
    print("########### Upsert new data #############")
    spark.sql("drop table if exists newData")
    spark.sql("CREATE TABLE if not exists newData(id LONG) USING iceberg location 's3a://hive/{tableName}'   ")
    spark.sql("truncate table newData")
    spark.sql("INSERT INTO newData VALUES 3, 4, 5, 6")

    spark.sql('''MERGE INTO {0} USING newData
            ON {0}.id = newData.id
            WHEN MATCHED THEN
              UPDATE SET {0}.id = newData.id
            WHEN NOT MATCHED THEN INSERT *
        '''.format(tableName))

    spark.sql("SELECT * FROM %s" % tableName).show()

    # Update table data
    print("########## Overwrite the table ###########")
    spark.sql("INSERT OVERWRITE %s select * FROM (VALUES 5, 6, 7, 8, 9) x (id)" % tableName)
    spark.sql("SELECT * FROM %s" % tableName).show()

    # Update every even value by adding 100 to it
    print("########### Update to the table(add 100 to every even value) ##############")
    spark.sql("UPDATE {0} SET id = (id + 100) WHERE (id % 2 == 0)".format(tableName))
    spark.sql("SELECT * FROM %s" % tableName).show()

    # Delete every even value
    print("######### Delete every even value ##############")
    spark.sql("DELETE FROM {0} WHERE (id % 2 == 0)".format(tableName))
    spark.sql("SELECT * FROM %s" % tableName).show()

    # Read old version of data using time travel
    print("######## Read old data using time travel ############")
    df = spark.read.format("delta").option("versionAsOf", 1).table(tableName)
    df.show()

finally:
    # cleanup
    print("fin")
    # spark.sql("select * from " + tableName ).show()
    #spark.sql("DROP TABLE IF EXISTS " + tableName)
    #spark.sql("DROP TABLE IF EXISTS newData")

############# Creating a table ###############
############ Reading the table ###############
+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+

########### Upsert new data #############
+---+
| id|
+---+
|  0|
|  1|
|  5|
|  6|
|  2|
|  3|
|  4|
+---+

########## Overwrite the table ###########
+---+
| id|
+---+
|  5|
|  6|
|  7|
|  8|
|  9|
+---+

########### Update to the table(add 100 to every even value) ##############
+---+
| id|
+---+
|  5|
|106|
|  7|
|108|
|  9|
+---+

######### Delete every even value ##############
+---+
| id|
+---+
|  7|
|  9|
|  5|
+---+

######## Read old data using time travel ############
+---+
| id|
+---+
|  7|
|  9|
|  5|
+---+

fin


In [5]:
spark.sql("UPDATE {0} SET id = (id + 100) WHERE (id % 2 == 1)".format(tableName))

DataFrame[]

In [6]:
spark.table(tableName).show()

+---+
| id|
+---+
|107|
|109|
|105|
+---+



DataFrame[]